## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key="g_key") 

In [26]:
# 1. Import the WeatherPy_database.csv file. 
cities_vacation_df = pd.read_csv("WeatherPy_database.csv")
cities_vacation_df.head()

,City ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Rain inches (last 1hrs),Snow inches (last 1hrs)
0,0,Naze,JP,2022-07-09 19:49:40,28.3667,129.4833,81.52,91,7,9.04,clear sky,0.0,0.0
1,1,Rocha,UY,2022-07-09 19:52:49,-34.4833,-54.3333,65.05,88,53,12.28,broken clouds,0.0,0.0
2,2,Barrow,US,2022-07-09 19:49:52,71.2906,-156.7887,53.62,76,100,10.36,overcast clouds,0.0,0.0
3,3,Vaini,TO,2022-07-09 19:49:41,-21.2000,-175.2000,73.56,83,40,9.22,scattered clouds,0.0,0.0
4,4,Kavaratti,IN,2022-07-09 19:51:57,10.5669,72.6420,81.16,82,95,21.03,overcast clouds,0.0,0.0


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
print("What is your minimum temperature preference?")
min_temp = float(input("min_temp: "))

print("What is your maximum temperature preference?")
max_temp = float(input("max_temp: "))

What is your minimum temperature preference?
min_temp: 70
What is your maximum temperature preference?
max_temp: 90


In [4]:
preffered_cities_df_1 = cities_vacation_df.loc[(cities_vacation_df["Max Temp"] <= max_temp) & (cities_vacation_df["Max Temp"] >= min_temp)]
preffered_cities_df_1.head()


,City ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Rain inches (last 1hrs),Snow inches (last 1hrs)
0,0,Naze,JP,2022-07-09 19:49:40,28.3667,129.4833,81.52,91,7,9.04,clear sky,0.0,0.0
3,3,Vaini,TO,2022-07-09 19:49:41,-21.2000,-175.2000,73.56,83,40,9.22,scattered clouds,0.0,0.0
4,4,Kavaratti,IN,2022-07-09 19:51:57,10.5669,72.6420,81.16,82,95,21.03,overcast clouds,0.0,0.0
8,8,Odweyne,SO,2022-07-09 19:56:03,9.4092,45.0640,71.02,54,100,20.65,overcast clouds,0.0,0.0
10,10,Arraial Do Cabo,BR,2022-07-09 19:50:11,-22.9661,-42.0278,76.96,88,40,14.97,scattered clouds,0.0,0.0


In [5]:
# 4a. Determine if there are any empty rows.
hotel_df = preffered_cities_df_1[["City","Country","Max Temp","Current Description","Lat","Lng",]].copy()
hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng
0,Naze,JP,81.52,clear sky,28.3667,129.4833
3,Vaini,TO,73.56,scattered clouds,-21.2000,-175.2000
4,Kavaratti,IN,81.16,overcast clouds,10.5669,72.6420
8,Odweyne,SO,71.02,overcast clouds,9.4092,45.0640
10,Arraial Do Cabo,BR,76.96,scattered clouds,-22.9661,-42.0278


In [15]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
drop_hotel_df = hotel_df.dropna()
drop_hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng
0,Naze,JP,81.52,clear sky,28.3667,129.4833
3,Vaini,TO,73.56,scattered clouds,-21.2000,-175.2000
4,Kavaratti,IN,81.16,overcast clouds,10.5669,72.6420
8,Odweyne,SO,71.02,overcast clouds,9.4092,45.0640
10,Arraial Do Cabo,BR,76.96,scattered clouds,-22.9661,-42.0278


In [16]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.

# 5b. Create a new column "Hotel Name"
drop_hotel_df["Hotel Name"] = ""
drop_hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Naze,JP,81.52,clear sky,28.3667,129.4833,
3,Vaini,TO,73.56,scattered clouds,-21.2000,-175.2000,
4,Kavaratti,IN,81.16,overcast clouds,10.5669,72.6420,
8,Odweyne,SO,71.02,overcast clouds,9.4092,45.0640,
10,Arraial Do Cabo,BR,76.96,scattered clouds,-22.9661,-42.0278,


In [17]:
#Import requests
import requests


from config import g_key

In [18]:
df_first_5 = drop_hotel_df.iloc[:5]

In [19]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.

for index, row in df_first_5.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame

    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        df_first_5.loc[index,"Hotel Name"] = hotels["results"][0]["name"]
    except(IndexError):
        print("Hotel not found.....")

In [20]:
# 7. Drop the rows where there is no Hotel Name.
df_first_5

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Naze,JP,81.52,clear sky,28.3667,129.4833,Shishime Hotel
3,Vaini,TO,73.56,scattered clouds,-21.2000,-175.2000,Keleti Beach Resort
4,Kavaratti,IN,81.16,overcast clouds,10.5669,72.6420,Green leaf
8,Odweyne,SO,71.02,overcast clouds,9.4092,45.0640,Eng Hiirey House
10,Arraial Do Cabo,BR,76.96,scattered clouds,-22.9661,-42.0278,Pousada Porto Praia


In [21]:
drop_hotel_df = drop_hotel_df[drop_hotel_df["Hotel Name"]!= '']
df_first_5

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Naze,JP,81.52,clear sky,28.3667,129.4833,Shishime Hotel
3,Vaini,TO,73.56,scattered clouds,-21.2000,-175.2000,Keleti Beach Resort
4,Kavaratti,IN,81.16,overcast clouds,10.5669,72.6420,Green leaf
8,Odweyne,SO,71.02,overcast clouds,9.4092,45.0640,Eng Hiirey House
10,Arraial Do Cabo,BR,76.96,scattered clouds,-22.9661,-42.0278,Pousada Porto Praia


In [22]:
# 8a. Create the output File (CSV)

# 8b. Export the City_Data into a csv
df_first_5.to_csv("weatherPy_vacation.csv", index_label="City_ID")

In [24]:
import gmaps
#Adding markers.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""
# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in df_first_5.iterrows()]

#Create the heatmap of temperature for the vacation spots and marker for each city.
locations = df_first_5[["Lat","Lng"]]
# max_temp = weather_vacation_df["Max Temp"]
fig=gmaps.figure() #(center=(30.0, 31.0), zoom_level=1.5)
# heat_layer=gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False, max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
# fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

#Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))